# New section

# Part 3: Data Analytics
0. Load both the csv file from Part 1 pr.data.0.Current and the json file from Part 2 as dataframes (Spark, Pyspark, Pandas, Koalas, etc).

1. Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

2. Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. For example, if the table had the following values:

  ```
  series_id	year	period	value
  PRS30006011	1995	Q01	1
  PRS30006011	1995	Q02	2
  PRS30006011	1996	Q01	3
  PRS30006011	1996	Q02	4
  PRS30006012	2000	Q01	0
  PRS30006012	2000	Q02	8
  PRS30006012	2001	Q01	2
  PRS30006012	2001	Q02	3
  ```
  
  the report would generate the following table:

  ```
  series_id	year	value
  PRS30006011	1996	7
  PRS30006012	2000	8
  ```

3. Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset). The below table shows an example of one row that might appear in the resulting table:

  ```
  series_id	year	period	value	Population
  PRS30006032	2018	Q01	1.9	327167439
  ```

- **Hints:** when working with public datasets you sometimes might have to perform some data cleaning first. For example, you might find it useful to perform trimming of whitespaces before doing any filtering or joins


4. Submit your analysis, your queries, and the outcome of the reports as a .ipynb file.

## Taks - 0. Data loading
Here I am loading the provided data files into DataFrames and using df.head() to show the data. I can use Pyspark dataframe too, but as the data voluem is small I am choosing pandas. *italicised text*


In [151]:
import pandas as pd

def load_and_clean_time_series_data(file_path='pr.data.0.Current', delimiter='\t'):
    try:
        time_series_df = pd.read_csv(file_path, delimiter=delimiter, keep_default_na=False)

        # cleaning column names using strip()
        time_series_df.columns = time_series_df.columns.str.strip()

        # cleaning 'series_id' column, as data also contains empty characters
        time_series_df['series_id'] = time_series_df['series_id'].str.strip()

        return time_series_df

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return  # Or raise the exception, depending on your needs.
    except pd.errors.ParserError:
        print(f"Error: Could not parse file '{file_path}'. Check delimiter and file format.")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return


# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #

time_series_df = load_and_clean_time_series_data()

if time_series_df is not None:
    print("\nCleaned DataFrame:")
    print(time_series_df.head())

    print("\nPrinting column names for testing purpose:")
    print("\nCleaned Column Names:", time_series_df.columns.tolist()) # display as list

    print("\nTesting datatype of various columns:")
    print(time_series_df.dtypes)



Cleaned DataFrame:
     series_id  year period  value footnote_codes
0  PRS30006011  1995    Q01    2.6               
1  PRS30006011  1995    Q02    2.1               
2  PRS30006011  1995    Q03    0.9               
3  PRS30006011  1995    Q04    0.1               
4  PRS30006011  1995    Q05    1.4               

Printing column names for testing purpose:

Cleaned Column Names: ['series_id', 'year', 'period', 'value', 'footnote_codes']

Testing datatype of various columns:
series_id          object
year                int64
period             object
value             float64
footnote_codes     object
dtype: object


In [136]:
import json

def load_and_display_population_data(file_path='population_data.json'):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)

        population_df = pd.DataFrame(data['data'])

        print("DataFrame Columns:")
        print(population_df.columns.tolist())  # Display columns as a list for clarity

        print("\nFirst 5 Rows of DataFrame:")
        print(population_df.head())

        return population_df  # Returning the DataFrame

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{file_path}'.")
    except KeyError as e:
        print(f"Error: Key '{e.args[0]}' not found in JSON data.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #

population_df = load_and_display_population_data()

DataFrame Columns:
['ID Nation', 'Nation', 'ID Year', 'Year', 'Population', 'Slug Nation']

First 5 Rows of DataFrame:
  ID Nation         Nation  ID Year  Year  Population    Slug Nation
0   01000US  United States     2023  2023   332387540  united-states
1   01000US  United States     2022  2022   331097593  united-states
2   01000US  United States     2021  2021   329725481  united-states
3   01000US  United States     2020  2020   326569308  united-states
4   01000US  United States     2019  2019   324697795  united-states


# Task 1. Generate the mean and the standard deviation of the annual US population across the years [2013, 2018] inclusive.

In [137]:
def calculate_population_stats(population_df, start_year=2013, end_year=2018):
    # filter dataFrame for given year range
    filtered_population_df = population_df[
        (population_df['Year'].astype(int) >= start_year) &
        (population_df['Year'].astype(int) <= end_year)
    ]


    mean_population = filtered_population_df['Population'].mean()
    std_population = filtered_population_df['Population'].std()

    # Printing results
    print(f"Mean population ({start_year}-{end_year}): {mean_population}")
    print(f"Standard deviation of population ({start_year}-{end_year}): {std_population}")

# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #


calculate_population_stats(population_df, 2013, 2018)  # Uses custom years


Mean population (2013-2018): 317437383.0
Standard deviation of population (2013-2018): 4257089.5415293295


In [138]:
def test_population_stats(population_df, start_year=2013, end_year=2018, tolerance=1e-6):
    filtered_df = population_df[
        (population_df['Year'].astype(int) >= start_year) &
        (population_df['Year'].astype(int) <= end_year)
    ]

    calculated_mean = filtered_df['Population'].mean()
    calculated_std = filtered_df['Population'].std()

    expected_mean = 317437383.0
    expected_std = 4257089.5415293295

    mean_within_tolerance = np.isclose(calculated_mean, expected_mean, atol=tolerance)
    std_within_tolerance = np.isclose(calculated_std, expected_std, atol=tolerance)

    print(f"Calculated Mean: {calculated_mean}, Expected Mean: {expected_mean}")
    print(f"Calculated Std Dev: {calculated_std}, Expected Std Dev: {expected_std}")
    print(f"Mean within tolerance: {mean_within_tolerance}")
    print(f"Std Dev within tolerance: {std_within_tolerance}")

    return mean_within_tolerance and std_within_tolerance

# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #

verification_result = test_population_stats(population_df)
print(f"Verification Result: {verification_result}")

Calculated Mean: 317437383.0, Expected Mean: 317437383.0
Calculated Std Dev: 4257089.5415293295, Expected Std Dev: 4257089.5415293295
Mean within tolerance: True
Std Dev within tolerance: True
Verification Result: True


# Task 2. For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year. For example, if the table had the following values:

In [132]:
import pandas as pd

def find_best_years(time_series_df):
    # convert 'value' to numeric and fill na with 0
    time_series_df['value'] = pd.to_numeric(time_series_df['value'], errors='coerce')
    time_series_df['value'] = time_series_df['value'].fillna(0)

    # calculate sum by year
    yearly_sums = time_series_df.groupby(['series_id', 'year'])['value'].sum().reset_index()

    # find best years (years with maximum sum for each series_id)
    best_years = yearly_sums.loc[yearly_sums.groupby('series_id')['value'].idxmax()]

    report_df = best_years[['series_id', 'year', 'value']]

    return report_df

# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #

report = find_best_years(time_series_df)
print(report)

        series_id  year    value
27    PRS30006011  2022   20.500
57    PRS30006012  2022   17.100
63    PRS30006013  1998  705.895
105   PRS30006021  2010   17.700
135   PRS30006022  2010   12.400
...           ...   ...      ...
8182  PRS88003192  2002  282.800
8233  PRS88003193  2023  847.203
8262  PRS88003201  2022   37.200
8292  PRS88003202  2022   28.700
8323  PRS88003203  2023  577.546

[282 rows x 3 columns]


In [139]:
def test_find_best_years(time_series_df, series_id='PRS88003192', year=2002):
    try:
        filtered_by_series = time_series_df[time_series_df['series_id'] == series_id]
        filtered_by_year = filtered_by_series[filtered_by_series['year'] == year]
        sum_of_values = filtered_by_year['value'].sum()
        print(f"Sum of values for year {year} (series_id = {series_id}): {sum_of_values}")

    except KeyError as e:
        print(f"Error: Column '{e.args[0]}' not found in DataFrame.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #

# verify_task2_result(time_series_df) #uses default values
# verify_task2_result(time_series_df, series_id='PRS88003192', year=2002) #example with explicit values.
test_find_best_result(time_series_df, series_id='PRS88003193', year=2023) #example with different values.

Sum of values for year 2023 (series_id = PRS88003193): 847.203


# 3. Using population and time_series data frame, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset). The below table shows an example of one row that might appear in the resulting table:

```series_id	year	period	value	Population
PRS30006032	2018	Q01	1.9	327167439
```


In [152]:
import pandas as pd

def series_value_and_population_q1(time_series_df, population_df):
    # Filter time_series_df for series_id and Q01
    filtered_time_series_df = time_series_df[
        (time_series_df['series_id'] == 'PRS30006032') &
        (time_series_df['period'] == 'Q01')
    ].copy()  # creating a copy as I was getting SettingWithCopyWarning

    # renaming column to lower case year
    population_df = population_df.rename(columns={'Year': 'year'})

    # join DataFrames on 'year'
    report_df = pd.merge(filtered_time_series_df, population_df, on='year', how='left')

    # Select desired columns for the report
    report_df = report_df[['series_id', 'year', 'period', 'value', 'Population']]

    return report_df

# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #

report = series_value_and_population_q1(time_series_df, population_df)
print(report)

      series_id  year period  value   Population
0   PRS30006032  1995    Q01    0.0          NaN
1   PRS30006032  1996    Q01   -4.2          NaN
2   PRS30006032  1997    Q01    2.8          NaN
3   PRS30006032  1998    Q01    0.9          NaN
4   PRS30006032  1999    Q01   -4.1          NaN
5   PRS30006032  2000    Q01    0.5          NaN
6   PRS30006032  2001    Q01   -6.3          NaN
7   PRS30006032  2002    Q01   -6.6          NaN
8   PRS30006032  2003    Q01   -5.7          NaN
9   PRS30006032  2004    Q01    2.0          NaN
10  PRS30006032  2005    Q01   -0.5          NaN
11  PRS30006032  2006    Q01    1.8          NaN
12  PRS30006032  2007    Q01   -0.8          NaN
13  PRS30006032  2008    Q01   -3.5          NaN
14  PRS30006032  2009    Q01  -21.0          NaN
15  PRS30006032  2010    Q01    3.2          NaN
16  PRS30006032  2011    Q01    1.5          NaN
17  PRS30006032  2012    Q01    2.5          NaN
18  PRS30006032  2013    Q01    0.5  311536594.0
19  PRS30006032  201

In [149]:
import pandas as pd

def test_series_value_and_population_q1(time_series_df, population_df):
    report_df = series_value_and_population_q1(time_series_df, population_df)
    report_df_2018 = report_df[report_df['year'] == 2018]

    # Expected output DataFrame
    expected_df = pd.DataFrame({
        'series_id': ['PRS30006032'],
        'year': [2018],
        'period': ['Q01'],
        'value': [0.5],  # Assuming this is the correct value for the test case
        'Population': [322903030.0]
    })

    # PRS30006032  2018    Q01    0.5  322903030.0
    # Check if the generated report matches the expected output, ignoring index
    try:
        pd.testing.assert_frame_equal(report_df_2018.reset_index(drop=True), expected_df, check_like=True)
        print("Test passed: Generated report matches expected output.")
        return True
    except AssertionError as e:
        print(f"Test failed: {e}")
        return False


# -------------------------------------------------------------------------- #
#                  Function Call Below This Line                             #
# -------------------------------------------------------------------------- #

test_result = test_series_value_and_population_q1(time_series_df, population_df)
print(f"Test Result: {test_result}")

      series_id  year period  value   Population
23  PRS30006032  2018    Q01    0.5  322903030.0
     series_id  year period  value   Population
0  PRS30006032  2018    Q01    0.5  322903030.0
Test passed: Generated report matches expected output.
Test Result: True
